In [ ]:
import duckdb
import fiona
from shapely.geometry import shape
from shapely.geometry import Polygon
import pandas as pd
import geopandas as gpd
from h3pandas.util.shapely import polyfill
import pyarrow.dataset as ds
import numpy as np
import re
import unicodedata

speciesgrids_path = "data/h3_5_obisgbif_20240723/"
shapefile_path = "data/marine_world_heritage_union.shp"
output_path = "output/lists.csv"

In [ ]:
def simplify_name(name):
    name = unicodedata.normalize("NFKD", name).encode("ascii", "ignore").decode("ascii").lower()
    name = re.sub(r"[^a-z0-9]", "_", name)
    name = re.sub(r"_+", "_", name)
    return name

def coords_to_polygon(coords):
    outer = coords[0]
    holes = coords[1:]
    polygon = Polygon(outer, holes)
    return polygon

In [ ]:
lists = []

con = duckdb.connect()
dataset = ds.dataset(speciesgrids_path, format="parquet")
con.register("dataset", dataset)

shapefile = fiona.open(shapefile_path)

for record in shapefile:

    site = record.properties.get("site")
    simplified = simplify_name(site)
    print(simplified)
    geometry = record["geometry"]
    geom_type = geometry["type"]

    polygons = list()

    if geom_type == "MultiPolygon":
        multi_polygon_coords = geometry["coordinates"]
        for polygon_coords in multi_polygon_coords:
            polygon = coords_to_polygon(polygon_coords)
            polygons.append(polygon)
    elif geom_type == "Polygon":
        polygon_coords = geometry["coordinates"]
        polygon = coords_to_polygon(polygon_coords)
        polygons.append(polygon)

    h3s = set()

    for polygon in polygons:
        h3 = list(polyfill(polygon, 5, geo_json=True))
        h3s.update(h3)

    con.register("cells", pd.DataFrame({"h3": list(h3s)}))
    df = con.execute(f"""
        select '{simplified}' as site, species, AphiaID, max(source_gbif) as source_gbif, max(source_obis) as source_obis, sum(records) as records, min(min_year) as min_year, max(max_year) as max_year
        from cells
        inner join dataset on cells.h3 = dataset.cell
        group by species, AphiaID
    """).fetchdf()

    lists.append(df)

full_list = pd.concat(lists)

In [8]:
full_list["min_year"] = full_list["min_year"].astype("Int64")
full_list["max_year"] = full_list["max_year"].astype("Int64")
full_list["records"] = full_list["records"].astype("Int64")

full_list.to_csv(output_path, index=False)

0       1849.0
1       1900.0
2       1929.0
3       1954.0
4       1973.0
         ...  
6086       NaN
6087    2014.0
6088       NaN
6089       NaN
6090    2014.0
Name: min_year, Length: 158630, dtype: float64